In [1]:
# Load models
from svc_helper.sfeatures.models import SVC5WhisperModel
from svc_helper.svc.rvc import RVCModel
from huggingface_hub import hf_hub_download

rvc_model = RVCModel()
test_model_path = hf_hub_download(repo_id='therealvul/RVCv2', 
    filename='RarityS1/Rarity.pth')
test_index_path = hf_hub_download(repo_id='therealvul/RVCv2', 
    filename='RarityS1/added_IVF1866_Flat_nprobe_1_Rarity_v2.index')
rvc_model.load_model(model_path = test_model_path,
    index_path = test_index_path)

whisper_model = SVC5WhisperModel()

2024-08-01 16:19:33 | WARNING | xformers | WARNING[XFORMERS]: xFormers can't load C++/CUDA extensions. xFormers was built for:
    PyTorch 2.1.0+cu118 with CUDA 1108 (you have 2.3.1+cu121)
    Python  3.10.11 (you have 3.10.7)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details
2024-08-01 16:19:33 | WARNING | xformers | A matching Triton is not available, some optimizations will not be enabled.
Error caught was: No module named 'triton'
2024-08-01 16:19:33 | WARNING | xformers | Triton is not available, some optimizations will not be enabled.
This is just a warning: triton is not available
2024-08-01 16:19:34 | INFO | faiss.loader | Loading faiss with AVX2 support.
2024-08-01 16:19:34 | INFO | faiss.loader | Could not load library with AVX2 support due to:
ModuleNotFoundError("No module named 'faiss.swigfaiss_avx2'")
202

is_half:True, device:cuda:0


2024-08-01 16:19:35 | INFO | svc_helper.svc.rvc.modules.vc.modules | Get sid: D:\hf_cache\hub\models--therealvul--RVCv2\snapshots\87778762d011892db45370f0dd963be836d55a08\RarityS1\Rarity.pth
2024-08-01 16:19:35 | INFO | svc_helper.svc.rvc.modules.vc.modules | Loading: D:\hf_cache\hub\models--therealvul--RVCv2\snapshots\87778762d011892db45370f0dd963be836d55a08\RarityS1\Rarity.pth
c:\Users\vul\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\utils\weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


In [2]:
from model import LitFeatureConvertor
checkpoint = './lightning_logs/version_0/checkpoints/epoch=18-step=9025.ckpt'
lmodel = LitFeatureConvertor.load_from_checkpoint(
    checkpoint,
    config={
        'train': {
            'lr': 3e-4
        },
        'model': {
            'in_emb_size': 1280,
            'out_emb_size': 768,
            'hidden_dim': 384,
            'dropout': 0.0,
            'n_layers': 6
        }
    }
)
lmodel.eval()

LitFeatureConvertor(
  (feature_convertor): FeatureConvertor(
    (in_proj): Linear(in_features=1280, out_features=384, bias=True)
    (blocks): Sequential(
      (0): Block(
        (ffn): Sequential(
          (0): Linear(in_features=384, out_features=1536, bias=True)
          (1): ReLU()
          (2): Linear(in_features=1536, out_features=384, bias=True)
          (3): Dropout(p=0.0, inplace=False)
        )
        (mamba): Mamba(
          (layers): ModuleList(
            (0): ResidualBlock(
              (mixer): MambaBlock(
                (in_proj): Linear(in_features=384, out_features=768, bias=False)
                (conv1d): Conv1d(384, 384, kernel_size=(4,), stride=(1,), padding=(3,), groups=384)
                (x_proj): Linear(in_features=384, out_features=56, bias=False)
                (dt_proj): Linear(in_features=24, out_features=384, bias=True)
                (out_proj): Linear(in_features=384, out_features=384, bias=False)
              )
              (norm): R

In [7]:
import IPython.display as ipd
in_file = '../3_basicdenoiser/ood5_male.wav'
transpose=15
index_rate=1.0

def feature_override(aud):
    feats = whisper_model.extract_features(aud).unsqueeze(0)
    return lmodel.infer(feats)

print('Unmodified')
wav_opt = rvc_model.infer_file(in_file, index_rate=index_rate,
    transpose=transpose)
ipd.display(ipd.Audio(wav_opt, rate=rvc_model.output_sample_rate()))
print('Whisper transform')
wav_opt = rvc_model.infer_file(in_file, index_rate=index_rate,
    transpose=transpose, extra_hooks={
        'feature_override': feature_override})
ipd.display(ipd.Audio(wav_opt, rate=rvc_model.output_sample_rate()))

Unmodified


Whisper transform
